In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from typing import TypedDict
from dotenv import load_dotenv


In [2]:
load_dotenv()

True

In [3]:
import os
os.environ.pop("GROQ_API_KEY", None)
print("GROQ:", os.getenv("GROQ_API_KEY"))

GROQ: None


In [4]:
os.environ.pop("GROQ_API_KEY", None)

In [5]:
llm = HuggingFaceEndpoint(
    repo_id="openai/gpt-oss-120b",
    task="text-generation"
)


/home/bumblebee/AgenticAI/venv/lib64/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
llm = ChatHuggingFace(llm=llm)

In [7]:
# create a state

class LLMState(TypedDict):

    question: str
    answer: str

In [8]:
def llm_qa(state: LLMState) -> LLMState:

    # extract the question from state
    question = state['question']

    # form a prompt
    prompt = f'Answer the following question {question}'

    # ask that question to the LLM
    answer = llm.invoke(prompt).content

    # update the answer in the state
    state['answer'] = answer

    return state

In [9]:
# create our graph
graph = StateGraph(LLMState)

# add nodes
graph.add_node('llm_qa', llm_qa)

# add edges
graph.add_edge(START, 'llm_qa')
graph.add_edge('llm_qa', END)

# compile
workflow = graph.compile()

In [16]:
# Execute
initial_state = {
    'question': 'Who won 2024 T20 WC and by how many runs? Who was Player of match? And give summary about final!!'
}
final_state = workflow.invoke(initial_state)  # Changed from 'intial_state' to 'initial_state'

print(final_state['answer'])

**2024 ICC Men’s T20 World Cup – Final**

| Item | Detail |
|------|--------|
| **Champion** | **India** |
| **Final margin** | **7 runs** (India 169/4 – South Africa 162/8) |
| **Player of the Match** | **Jasprit Bumrah** (4 wickets for 19 runs, plus 15 runs off 9 balls) |
| **Date & Venue** | 29 June 2024 – Nassau County International Cricket Stadium, New York, USA |

---

### Quick‑hit summary of the final

1. **Toss & Strategy**  
   - South Africa won the toss and elected to field, hoping to chase down whatever India posted on a relatively flat New York pitch.

2. **India’s innings – 20 overs, 169/4**  
   - **Openers:** Rohit Sharma (48 off 38) and Shubman Gill (33 off 24) gave India a steady start, taking the score to 78/0 after 9 overs.  
   - **Middle order:** Virat Kohli (52 off 37) accelerated, reaching his 60‑run milestone with a couple of sixes.  
   - **Finisher:** Hardik Pandya (27 off 13) added a quick burst, and the tail‑ender Jasprit Bumrah sealed the total with a bou